<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [546]:
import pandas as pd
import psycopg2

In [547]:
# Отключение предупреждения (оно не мешает работе, но отвлекает и занимает место):
# "UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) 
# or database string URI or sqlite3 DBAPI2 connection. 
# Other DBAPI2 objects are not tested. Please consider using SQLAlchemy".
import warnings
warnings.filterwarnings('ignore')

In [548]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [549]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [550]:
# текст запроса
query_3_1 = f'''
select *
from public.vacancies
'''

In [551]:
# результат запроса
df_3_1 = pd.read_sql_query(query_3_1, connection)
print(f'Количество вакансий в таблице "Vacancies": {df_3_1.shape[0]}')

Количество вакансий в таблице "Vacancies": 49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [552]:
# текст запроса
query_3_2 = f'''
select *
from public.employers
'''

In [553]:
# результат запроса
df_3_2 = pd.read_sql_query(query_3_2, connection)
print(f'Количество работодателей в таблице "Employers": {df_3_2.shape[0]}')

Количество работодателей в таблице "Employers": 23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [554]:
# текст запроса
query_3_3 = f'''
select *
from public.areas
'''

In [555]:
# результат запроса
df_3_3 = pd.read_sql_query(query_3_3, connection)
print(f'Количество регионов в таблице "Areas": {df_3_3.shape[0]}')

Количество регионов в таблице "Areas": 1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [556]:
# текст запроса
query_3_4 = f'''
select *
from public.industries
'''

In [557]:
# результат запроса
df_3_4 = pd.read_sql_query(query_3_4, connection)
print(f'Количество сфер деятельности в таблице "Industries": {df_3_4.shape[0]}')

Количество сфер деятельности в таблице "Industries": 294


***

In [558]:
# выводы по предварительному анализу данных

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [559]:
# текст запроса
query_4_1 = f'''
select
    a.name area,
    count(v.area_id) cnt
from public.areas a
    join public.vacancies v on a.id = v.area_id
group by area
order by count(v.area_id) desc
limit 5
'''

In [600]:
# результат запроса
df_4_1 = pd.read_sql_query(query_4_1, connection)
print('Топ 5 регионов по количеству вакансий:')
display(df_4_1)

Топ 5 регионов по количеству вакансий:


,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [601]:
# текст запроса
query_4_2 = f'''
select
    count(id)
from public.vacancies
where (salary_from is not null) or (salary_to is not null)
'''

In [602]:
# результат запроса
df_4_2 = pd.read_sql_query(query_4_2, connection)
print(f'Количество вакансий, у которых заполнено хотя бы одно из двух \
полей с зарплатой: {df_4_2.iloc[0,0]}')

Количество вакансий, у которых заполнено хотя бы одно из двух полей с зарплатой: 24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [603]:
# текст запроса
query_4_3 = f'''
select
    round(avg(salary_from), 0) avg_salart_from,
    round(avg(salary_to), 0) avg_salart_to
from public.vacancies
'''

In [564]:
# результат запроса
df_4_3 = pd.read_sql_query(query_4_3, connection)
print(f'Средние значения для нижней {df_4_3.iloc[0,0]} и верхней {df_4_3.iloc[0,1]} границ зарплатной вилки')
display(df_4_3)


Средние значения для нижней 71065.0 и верхней 110537.0 границ зарплатной вилки


,avg_salart_from,avg_salart_to
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [604]:
# текст запроса
query_4_4 = f'''
select
    schedule || ' - ' || employment as "тип грвфика - тип трудоустройства",
    count(id) "кол-во вакансий"
from public.vacancies
group by "тип грвфика - тип трудоустройства"
order by 2 desc
offset 1
limit 1
'''

In [566]:
# результат запроса
df_4_4 = pd.read_sql_query(query_4_4, connection)
print(f'Второе месте по популярности пары "тип грвфика - тип трудоустройства": \
{df_4_4.iloc[0,0]}')
display(df_4_4)

Второе месте по популярности пары "тип грвфика - тип трудоустройства": Удаленная работа - Полная занятость


,тип грвфика - тип трудоустройства,кол-во вакансий
0,Удаленная работа - Полная занятость,7802


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [605]:
# текст запроса
query_4_5 = f'''
select
    experience, 
    count(id) count
from public.vacancies
group by  experience
order by 2
'''

In [606]:
# результат запроса
df_4_5 = pd.read_sql_query(query_4_5, connection)
print(f'Требуемый опыт работы (experience) в порядке возрастания количества вакансий:')
display(df_4_5)

Требуемый опыт работы (experience) в порядке возрастания количества вакансий:


,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

In [569]:
# выводы по детальному анализу вакансий

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [607]:
# текст запроса
query_5_1 = f'''
select
    e.name empl_name,
    count(v.id) count
from public.employers e
    join public.vacancies v on e.id = v.employer_id
group by  empl_name
order by 2 desc
limit 5
'''

In [608]:
# результат запроса
df_5_1 = pd.read_sql_query(query_5_1, connection)
print(f'Работодатели, находятся на первом ({df_5_1.iloc[0][0]}) \
и пятом ({df_5_1.iloc[-1][0]}) месте по количеству вакансий')
display(df_5_1)

Работодатели, находятся на первом (Яндекс) и пятом (Газпром нефть) месте по количеству вакансий


,empl_name,count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [609]:
# текст запроса
query_5_2 = f'''
select
    e.name empl_name,
    count(v.id) count
from public.employers e
    join public.vacancies v on e.id = v.employer_id
group by  empl_name
order by 2 desc
limit 5
'''

In [610]:
# результат запроса
df_5_2 = pd.read_sql_query(query_5_2, connection)
#print(f'Работодатели, находятся на первом ({df_5_1.iloc[0][0]}) \
#и пятом ({df_5_1.iloc[-1][0]}) месте по количеству вакансий')
display(df_5_2)

,empl_name,count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [574]:
# текст запроса

In [575]:
# результат запроса

4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [576]:
# текст запроса

In [577]:
# результат запроса

5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [578]:
# текст запроса

In [579]:
# результат запроса

6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [580]:
# текст запроса

In [581]:
# результат запроса

7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [582]:
# код для получения списка городов-милионников

In [583]:
# текст запроса

In [584]:
# результат запроса

***

In [585]:
# выводы по анализу работодателей

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [586]:
# текст запроса

In [587]:
# результат запроса

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [588]:
# текст запроса

In [589]:
# результат запроса

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [590]:
# текст запроса

In [591]:
# результат запроса

4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [592]:
# текст запроса

In [593]:
# результат запроса

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [594]:
# текст запроса

In [595]:
# результат запроса

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [596]:
# текст запроса

In [597]:
# результат запроса

***

In [598]:
# выводы по предметному анализу

# Общий вывод по проекту

In [599]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования